In [1]:
''' 
Usage:
    python manage.py shell
    >> execfile('create_test_data.py')
'''
from django.db.models.query_utils import DeferredAttribute

from tracker.models import (
    Profile, Exercise, ExerciseInstance,
    Classification, Set, MuscleGroup
)

import pandas as pd
from IPython.display import display, HTML


In [2]:
class InstanceSetDict(object):
    ''' Transform an ExerciseInstance objects set data into a dict.'''
    
    def __init__(self, instance):
        self.instance = instance
        self.sets = instance.sets.all()
        self.attributes = self._get_all_set_attr()
        self.dict = {}
        self._create_dict()

    def _get_all_set_attr(self):
        ''' Get Set attributes of interest and return as a list.'''
        set_attr_keys = []
        for k, i in Set.__dict__.items():
            if isinstance(i, DeferredAttribute) and k != 'id':
                set_attr_keys.append(k.replace('_', ''))
        return set_attr_keys
    
    def _add_header(self):
        ''' Add set attributes as keys to the table dict.'''
        self.dict = {k.capitalize(): [] for k in self.attributes}
    
    def _get_set_value(self, aset, attr):
        ''' Get the value of a given Set objects attribute.'''
        set_value = getattr(aset, attr)
        if set_value:
            if attr in ['weight', 'distance']:
                # get participants preffered unit of measurement and return value in said unit
                unit = getattr(self.instance.participant, f'{attr}_unit')
                set_value = getattr(set_value, unit)
        else:
            set_value = None       
        return set_value
        
    def _add_set_data(self):
        ''' Add all sets data to the dictionary'''
        for aset in self.sets:
            for attr in self.attributes:
                set_value = self._get_set_value(aset, attr)
                self.dict[attr.capitalize()].append(set_value)
    
    def _create_dict(self):
        self._add_header()
        self._add_set_data()
        
    def to_dataframe(self):
        df = pd.DataFrame.from_dict(self.dict)
        df = df[df.columns[~df.isnull().all()]]
        df['Sets'] = df.index + 1
        df['Date'] = self.instance.date
        df['Exercise'] = self.instance.exercise.name
        df = df.set_index(['Exercise', 'Date', 'Sets'])
        return df
    
    def to_html(self):
        df = self.to_dataframe()
        return pd.DataFrame.to_html(df)
        


instance = ExerciseInstance.objects.first()
d = InstanceSetDict(instance)
d.to_dataframe()

Reps  Weight
Exercise Date       Sets              
Pull Up  2018-09-09 1       12    20.0
                    2       10    20.0

In [8]:
import datetime


def exercise_instance_table(qs):
    ''' Create a HTML table from a given ExerciseInstance QuerySet'''
    # Get all instance by Exercise Type
    all_dfs = []
    for instance in qs:
        df = InstanceSetDict(instance).to_dataframe()
        all_dfs.append(df)


    big_table = pd.concat(all_dfs, sort=False).T
    return big_table.to_html()

qs = ExerciseInstance.objects.filter(exercise__name='Pull Up')
qs = ExerciseInstance.objects.filter(date=datetime.date(2018, 10, 9))
qs = ExerciseInstance.objects.filter(exercise__classification__name='Push')

exercise_instance_table(qs)

'<table border="1" class="dataframe">\n  <thead>\n    <tr>\n      <th>Exercise</th>\n      <th colspan="2" halign="left">Push Up</th>\n      <th colspan="3" halign="left">Dip</th>\n    </tr>\n    <tr>\n      <th>Date</th>\n      <th colspan="2" halign="left">2018-10-09</th>\n      <th colspan="3" halign="left">2018-11-09</th>\n    </tr>\n    <tr>\n      <th>Sets</th>\n      <th>1</th>\n      <th>2</th>\n      <th>1</th>\n      <th>2</th>\n      <th>3</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Reps</th>\n      <td>50</td>\n      <td>40</td>\n      <td>15</td>\n      <td>14</td>\n      <td>12</td>\n    </tr>\n  </tbody>\n</table>'